In [10]:
from scipy import optimize
from scipy.stats import norm
import pandas as pd
import numpy as np

brentq = optimize.brentq
N = norm.cdf


def IV(S0,K,r,T,V,sigma):
    d1 = (np.log(S0/K) + (r + 0.5 * (sigma ** 2)) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S0/K) + (r - 0.5 * (sigma ** 2)) * T) / (sigma * np.sqrt(T))
    return S0 * N(d1) - K * np.exp(-r*T) * N(d2) - V

def IV_solver(S0, K, r, T, V):

    def IV_brentq(sigma):
        return IV(S0,K,r,T,V,sigma) 
    
    return brentq(IV_brentq, a=0, b=1) 

IV_solver(S0=1,K=1/0.8,r=0.03,T=1,V=0.2)

<ipython-input-10-74b272ac9257>:11: RuntimeWarning: divide by zero encountered in double_scalars
  d1 = (np.log(S0/K) + (r + 0.5 * (sigma ** 2)) * T) / (sigma * np.sqrt(T))
<ipython-input-10-74b272ac9257>:12: RuntimeWarning: divide by zero encountered in double_scalars
  d2 = (np.log(S0/K) + (r - 0.5 * (sigma ** 2)) * T) / (sigma * np.sqrt(T))


0.6818194650870998

In [11]:
    def simulate(self):

        model = self.model 
        Type = self.Type 
        option = self.option

        American = False if Type == 'European' else True
        optionType = 'c' if option == 'call' else 'p'

        if model == 'GBM':
            cols = ['moneyness', 'tau', 'r', 'vol', 'option price']
            data = []
            for i in tqdm(range(10000)):
                moneyness = random.uniform(0.5, 1.5)
                tau = random.uniform(0.3, 0.95)
                r = random.uniform(0.03, 0.08)
                vol = random.uniform(0.02, 0.9)
                # log_moneyness = np.log( moneyness ) 
                MC = MonteCarlo(model='GBM', S0=1, K=1/moneyness, T=tau, r=r, q=0, v=vol, method='MC')
                MC.generate_S(1000,10)
                option_price = MC.pricer(optionType=optionType, American=American)
                IV = IV_solver(S0=1,K=1/moneyness,r=r,T=tau,V=option_price)
                data.append([moneyness, tau, r, vol, option_price,IV])
        
        elif model == 'Heston':
            cols = ['moneyness', 'tau', 'r', 'vol0', 'theta', 'kappa', 'gamma', 'rho', 'option price']
            data = []
            for i in tqdm(range(10000)):
                moneyness = random.uniform(0.6, 1.4)
                tau = random.uniform(0.1, 1.4)
                r = random.uniform(0, 0.1)
                vol0 = random.uniform(0.05, 0.5)
                theta = random.uniform(0, 0.5)
                kappa = random.uniform(0, 2)
                gamma = random.uniform(0, 0.5)
                rho = random.uniform(-0.95, 0)
                # moneyness = np.log( moneyness )                      
                MC = MonteCarlo(model='Heston', S0=1, K=1/moneyness, T=tau, r=r, q=0, v0=vol0, theta=theta, kappa=kappa, gamma=gamma, rho=rho)
                MC.generate_S(1000,10)
                option_price = MC.pricer(optionType=optionType, American=American)
                IV = IV_solver(S0=1,K=1/moneyness,r=r,T=tau,V=option_price)
                data.append([moneyness, tau, r, vol0, theta, kappa, gamma, rho, option_price, IV])

        else: 
            raise(ValueError(f'You input a not-supported model type: {model}'))

        data = pd.DataFrame(data, columns=cols)
        data.to_csv(f'data_{model}_{Type}_{option}.csv', index=False)